# Load GLEIF Data from public repository on AWS (s3://gleif/)
Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
# pip install boto3
# pip install python-dotenv
# pip install trino
# pip install pandas

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import pandas as pd
import trino
import io

In [3]:
# Use unsigned access to public S3 resource for GLEIF bucket
s3_resource = boto3.resource(
    service_name="s3",
    config=Config(signature_version=UNSIGNED),
)
bucket = s3_resource.Bucket("gleif")

In [4]:
# Retrieve latest LEI data set
LEI_data_sets = []
for obj in bucket.objects.filter(Prefix='data/csv/LEI/'):
    LEI_data_sets.append(obj.key)
LEI_data_sets[len(LEI_data_sets)-1]

'data/csv/LEI/date=2021-08-27/time=16:00/20210827-1600-gleif-goldencopy-lei2-golden-copy.csv.gz'

In [ ]:
# LEI_data_set_to_ingest = s3_resource.Object("gleif", LEI_data_sets[len(LEI_data_sets)-1])
# df = (pd.read_csv(obj['Body'])).convert_dtypes()